In [5]:
import sys
import requests
import json
import time
import pandas as pd

In [7]:
file_location='C:/Users/GOWRICH1/Downloads/Sandoz_Intra_cert/Sandoz_Intra_cert/Dev_cert.pem'
def get_new_token():
    auth_server_url = "https://api-dev-sandoz.eu.novartis.net/api/orbit/sandoz/oauth/token"
    client_id = '76927862-a359-4470-bffe-6c8b77090d93'
    client_secret = 'aa1ad6e8-cd87-48b2-b1a0-863ccdcc223e'

    token_req_payload = {'grant_type': 'client_credentials'}

    token_response = requests.post(auth_server_url,
    data=token_req_payload, verify=file_location, allow_redirects=False,
    auth=(client_id, client_secret))

    if token_response.status_code !=200:
        print("Failed to obtain token from the OAuth 2.0 server")
        sys.exit(1)

    print("Successfuly obtained a new token")
    tokens = json.loads(token_response.text)
    return tokens['access_token']

In [8]:
def download_data(condition):   
    condition = condition.lower().replace(" ","+")
    try:
        token = get_new_token()
        api_call_headers = {'Authorization': 'Bearer ' + token}
        request1=requests.get('https://api-dev-sandoz.eu.novartis.net/api/1.0/orbit/sandoz/query/full_studies?expr=%s+AND+AREA[PrimaryCompletionDate]RANGE[01/01/2022,MAX]+AND(AREA[OverallStatus]Recruiting+OR+AREA[OverallStatus]Not+yet+recruiting+OR+AREA[OverallStatus]Active,+not+recruiting+OR+AREA[OverallStatus]Enrolling+by+invitation+OR+AREA[OverallStatus]Unknown+status)&min_rnk=1&max_rnk=100&fmt=json'%condition,headers=api_call_headers, verify=file_location)
        time.sleep(10)
        elevations = request1.text
        data = json.loads(elevations)
        total_count = data['FullStudiesResponse']['NStudiesFound']
        all_data = pd.json_normalize(data['FullStudiesResponse']['FullStudies'])
        print('100 studies updated')
        for i in range(total_count//100+1):
            if(i!=0):
                request2=requests.get('https://api-dev-sandoz.eu.novartis.net/api/1.0/orbit/sandoz/query/full_studies?expr=%s+AND+AREA[PrimaryCompletionDate]RANGE[01/01/2022,MAX]+AND(AREA[OverallStatus]Recruiting+OR+AREA[OverallStatus]Not+yet+recruiting+OR+AREA[OverallStatus]Active,+not+recruiting+OR+AREA[OverallStatus]Enrolling+by+invitation+OR+AREA[OverallStatus]Unknown+status)&min_rnk=%d&max_rnk=%d&fmt=json'%(condition,i*100,i*100+99),headers=api_call_headers, verify=file_location)
                time.sleep(10)
                elevations = request2.text
                data = json.loads(elevations)
                all_data = all_data.append(pd.json_normalize(data['FullStudiesResponse']['FullStudies']))
                print('%d studies updated'%(i*100+100))
        all_data['disease'] =  condition.lower().replace("+"," ").strip()   
        return all_data
    except Exception as e:
        print('Failed to fetch data for',condition.lower().replace("+"," ").strip())
        print(e)

In [9]:
download_data('asthma')

Successfuly obtained a new token
100 studies updated


KeyboardInterrupt: 